In [1]:
from datasets.get_datasets import *
from revision import *
from boostsrl import boostsrl
import random
from sklearn.model_selection import KFold

## Parameters

In [2]:
target = 'athleteplaysforteam'
#test_size = 0.3
small_train_size = 0.3
validation_size = 0.2
revision_threshold = 0.95
max_revision_iterations = 10

## Dataset

In [3]:
[facts, pos, neg] = get_nell_dataset(target)

## Background  configuration

In [4]:
bk = ['athleteledsportsteam(+athlete,+sportsteam).',
      'athleteledsportsteam(+athlete,-sportsteam).',
      'athleteledsportsteam(-athlete,+sportsteam).',
      'athleteplaysforteam(+athlete,+sportsteam).',
      'athleteplaysforteam(+athlete,-sportsteam).',
      'athleteplaysforteam(-athlete,+sportsteam).',
      'athleteplaysinleague(+athlete,+sportsleague).',
      'athleteplaysinleague(+athlete,-sportsleague).',
      'athleteplaysinleague(-athlete,+sportsleague).',
      'athleteplayssport(+athlete,+sport).',
      'athleteplayssport(+athlete,-sport).',
      'athleteplayssport(-athlete,+sport).',
      'teamalsoknownas(+sportsteam,+sportsteam).',
      'teamalsoknownas(+sportsteam,-sportsteam).',
      'teamalsoknownas(-sportsteam,+sportsteam).',
      'teamplaysagainstteam(+sportsteam,+sportsteam).',
      'teamplaysagainstteam(+sportsteam,-sportsteam).',
      'teamplaysagainstteam(-sportsteam,+sportsteam).',
      'teamplaysinleague(+sportsteam,+sportsleague).',
      'teamplaysinleague(+sportsteam,-sportsleague).',
      'teamplaysinleague(-sportsteam,+sportsleague).',
      'teamplayssport(+sportsteam,+sport).',
      'teamplayssport(+sportsteam,-sport).',
      'teamplayssport(-sportsteam,+sport).']

background = boostsrl.modes(bk, [target], useStdLogicVariables=False, treeDepth=8, nodeSize=3, numOfClauses=8)

## Sampling examples

In [5]:
# shuffle all examples
random.shuffle(pos)
random.shuffle(neg)

neg = neg[:len(pos)] # balanced

import numpy as np
pos = np.array(pos)
neg = np.array(neg)
small_dataset_aucroc = []
complete_dataset_aucroc = []
revision_dataset_aucroc = []
small_learning_time = []
small_inference_time = []
complete_learning_time = []
complete_inference_time = []
revision_learning_time = []
revision_inference_time = []

# separate train and test
kf = KFold(n_splits=10)
for train_index, test_index in kf.split(pos):
    train_pos, test_pos = pos[train_index], pos[test_index]
    train_neg, test_neg = neg[train_index], neg[test_index]

    # learn from scratch in a small dataset
    s_train_pos = train_pos[:int(small_train_size*len(train_pos))]
    s_train_neg = train_neg[:int(small_train_size*len(train_neg))]

    # shuffle all train examples
    random.shuffle(train_pos)
    random.shuffle(train_neg)

    # train set used in revision and validation set
    r_train_pos = train_pos[int(validation_size*len(train_pos)):]
    r_train_neg = train_neg[int(validation_size*len(train_neg)):]
    validation_pos = train_pos[:int(validation_size*len(train_pos))]
    validation_neg = train_neg[:int(validation_size*len(train_neg))]

    print('Total examples')
    print('Positive examples: %s' % len(pos))
    print('Negative examples: %s' % len(neg))
    print('\n')
    print('Train and test sets from total')
    print('Train Positive examples: %s' % len(train_pos))
    print('Train Negative examples: %s' % len(train_neg))
    print('Test Positive examples: %s' % len(test_pos))
    print('Test Negative examples: %s' % len(test_neg))
    print('\n')
    print('Small dataset')
    print('Train Positive examples: %s' % len(s_train_pos))
    print('Train Negative examples: %s' % len(s_train_neg))
    print('\n')
    print('Revision theory train dataset and validation')
    print('Train Positive examples: %s' % len(r_train_pos))
    print('Train Negative examples: %s' % len(r_train_neg))
    print('Validation Positive examples: %s' % len(validation_pos))
    print('Validation Negative examples: %s' % len(validation_neg))
    
    # learning from small dataset
    delete_model_files()
    model = boostsrl.train(background, s_train_pos, s_train_neg, facts)
    learning_time = model.traintime()
    will = model.get_will_produced_tree()
    structured = model.get_structured_tree().copy()
    results = boostsrl.test(model, test_pos, test_neg, facts)
    inference_time = results.testtime()
    t_results = results.summarize_results()
    print('WILL-Produced Tree:')
    print_will_produced_tree(will)
    print('\n')
    print('Results:')
    print(t_results)
    print('\n')
    print('Total learning time: %s seconds' % learning_time)
    print('Total inference time: %s seconds' % inference_time)
    print('AUC ROC: %s' % t_results['AUC ROC'])
    print('\n')
    #print('Tree:')
    #model.tree(0, target, image=True)
    small_dataset_aucroc.append(t_results['AUC ROC'])
    small_learning_time.append(learning_time)
    small_inference_time.append(inference_time)
    
    # learning from complete dataset
    delete_model_files()
    model = boostsrl.train(background, train_pos, train_neg, facts)
    learning_time = model.traintime()
    will = model.get_will_produced_tree()
    #structured = model.get_structured_tree().copy()
    results = boostsrl.test(model, test_pos, test_neg, facts)
    inference_time = results.testtime()
    t_results = results.summarize_results()
    print('WILL-Produced Tree:')
    print_will_produced_tree(will)
    print('\n')
    print('Results:')
    print(t_results)
    print('\n')
    print('Total learning time: %s seconds' % learning_time)
    print('Total inference time: %s seconds' % inference_time)
    print('AUC ROC: %s' % t_results['AUC ROC'])
    print('\n')
    #print('Tree:')
    #model.tree(0, target, image=True)
    complete_dataset_aucroc.append(t_results['AUC ROC'])
    complete_learning_time.append(learning_time)
    complete_inference_time.append(inference_time)
    
    # theory revision
    total_revision_time = 0
    best_aucroc = 0
    best_structured = None

    print('Performing parameter learning')
    print('******************************************')
    delete_model_files()
    model = boostsrl.train(background, r_train_pos, r_train_neg, facts, refine=get_refine_file(structured))
    learning_time = model.traintime()
    will = model.get_will_produced_tree()
    structured = model.get_structured_tree().copy()
    results = boostsrl.test(model, validation_pos, validation_neg, facts)
    inference_time = results.testtime()
    t_results = results.summarize_results()
    total_revision_time += learning_time + inference_time
    print('WILL-Produced Tree:')
    print_will_produced_tree(will)
    print('\n')
    print('Results:')
    print(t_results)
    print('\n')
    print('Total learning time: %s seconds' % learning_time)
    print('Total inference time: %s seconds' % inference_time)
    print('AUC ROC: %s' % t_results['AUC ROC'])
    print('\n')
    #print('Tree:')
    #model.tree(0, target, image=True)

    best_aucroc = t_results['AUC ROC']
    best_structured = structured.copy()
    save_model_files()

    for i in range(max_revision_iterations):
        print('Refining iteration %s' % str(i+1))
        print('******************************************')
        found_better = False
        for candidate in get_cantidates(best_structured, revision_threshold):
            print('Refining node candidate')
            print('******************************************')
            delete_model_files()
            model = boostsrl.train(background, r_train_pos, r_train_neg, facts, refine=candidate)
            learning_time = model.traintime()
            will = model.get_will_produced_tree()
            results = boostsrl.test(model, validation_pos, validation_neg, facts)
            inference_time = results.testtime()
            t_results = results.summarize_results()
            total_revision_time += learning_time + inference_time
            print('WILL-Produced Tree:')
            print_will_produced_tree(will)
            print('\n')
            print('Results:')
            print(t_results)
            print('\n')
            print('Total learning time: %s seconds' % learning_time)
            print('Total inference time: %s seconds' % inference_time)
            print('AUC ROC: %s' % t_results['AUC ROC'])
            print('\n')
            #print('Tree:')
            #model.tree(0, target, image=True)
            print('******************************************')

            if t_results['AUC ROC'] > best_aucroc:
                found_better = True
                best_aucroc = t_results['AUC ROC']
                best_structured = model.get_structured_tree().copy()
                save_model_files()
        print('Best Tree AUC ROC so far: %s' % best_aucroc)
        print('******************************************\n')
        if found_better == False:
            break

    print('******************************************')
    delete_model_files()
    get_saved_model_files()
    delete_test_files()
    print('Total revision time: %s' % total_revision_time)
    print('Best validation AUC ROC: %s' % best_aucroc)
    will = model.get_will_produced_tree()
    results = boostsrl.test(model, test_pos, test_neg, facts)
    inference_time = results.testtime()
    t_results = results.summarize_results()
    print('WILL-Produced Tree:')
    print_will_produced_tree(will)
    print('\n')
    print('Results:')
    print(t_results)
    print('\n')
    print('Total inference time: %s seconds' % inference_time)
    print('AUC ROC: %s' % t_results['AUC ROC'])
    print('\n')
    #print('Tree:')
    #model.tree(0, target, image=True)
    revision_dataset_aucroc.append(t_results['AUC ROC'])
    revision_learning_time.append(total_revision_time)
    revision_inference_time.append(inference_time)

Total examples
Positive examples: 1458
Negative examples: 1458


Train and test sets from total
Train Positive examples: 1312
Train Negative examples: 1312
Test Positive examples: 146
Test Negative examples: 146


Small dataset
Train Positive examples: 393
Train Negative examples: 393


Revision theory train dataset and validation
Train Positive examples: 1050
Train Negative examples: 1050
Validation Positive examples: 262
Validation Negative examples: 262
WILL-Produced Tree:
% FOR athleteplaysforteam(A, B):
%   if ( teamplaysagainstteam(B, C), athleteledsportsteam(D, C), athleteplaysinleague(D, E) )
%   then if ( athleteledsportsteam(A, B) )
%   | then return 0.8581489350995127;  // std dev = 0,000, 70,000 (wgt'ed) examples reached here.  /* #pos=70 */
%   | else if ( athleteledsportsteam(A, F), teamplaysagainstteam(G, F), teamalsoknownas(H, G) )
%   | | then if ( teamplaysagainstteam(F, B), teamplaysinleague(F, E), athleteplaysinleague(A, E) )
%   | | | then return 0.6228548174524535

In [6]:
small_dataset_aucroc

[0.733557,
 0.74552,
 0.756943,
 0.750446,
 0.819877,
 0.814271,
 0.791753,
 0.816664,
 0.809132,
 0.816361]

In [7]:
complete_dataset_aucroc

[0.710593,
 0.757201,
 0.774512,
 0.75258,
 0.803082,
 0.791143,
 0.79384,
 0.802965,
 0.831843,
 0.818668]

In [8]:
revision_dataset_aucroc

[0.731493,
 0.744863,
 0.757834,
 0.788539,
 0.818376,
 0.812301,
 0.791753,
 0.822434,
 0.814221,
 0.824518]

In [9]:
print('Small dataset AUC ROC: %s +/- %s' % ((np.array(small_dataset_aucroc)).mean(), (np.array(small_dataset_aucroc)).std()))
print('Complete dataset AUC ROC: %s +/- %s' % ((np.array(complete_dataset_aucroc)).mean(), (np.array(complete_dataset_aucroc)).std()))
print('Theory revision dataset AUC ROC: %s +/- %s' % ((np.array(revision_dataset_aucroc)).mean(), (np.array(revision_dataset_aucroc)).std()))
print('Small dataset learning time: %s +/- %s' % ((np.array(small_learning_time)).mean(), (np.array(small_learning_time)).std()))
print('Small dataset inference time: %s +/- %s' % ((np.array(small_inference_time)).mean(), (np.array(small_inference_time)).std()))
print('Complete dataset learning time: %s +/- %s' % ((np.array(complete_learning_time)).mean(), (np.array(complete_learning_time)).std()))
print('Complete dataset inference time: %s +/- %s' % ((np.array(complete_inference_time)).mean(), (np.array(complete_inference_time)).std()))
print('Revision time: %s +/- %s' % ((np.array(revision_learning_time)).mean(), (np.array(revision_learning_time)).std()))
print('Revision model inference time: %s +/- %s' % ((np.array(revision_inference_time)).mean(), (np.array(revision_inference_time)).std()))

Small dataset AUC ROC: 0.7854523999999999 +/- 0.03297188859680318
Complete dataset AUC ROC: 0.7836427 +/- 0.03397822954790315
Theory revision dataset AUC ROC: 0.7906332 +/- 0.03260201371019894
Small dataset learning time: 61.31640000000001 +/- 48.34885805104398
Small dataset inference time: 0.9378 +/- 0.09096460850242802
Complete dataset learning time: 50.5578 +/- 26.638979724456416
Complete dataset inference time: 0.8493999999999999 +/- 0.0745831080071084
Revision time: 34.7254 +/- 15.87966005429587
Revision model inference time: 0.9875999999999999 +/- 0.12993629208192758
